## Practical basics

It is easy to calculate the quantities we encounter most often in statistics!

# Very WIP!

Basic statistics - practical skills
- Maybe start with counting and summing?
- For the card experiment, how to calculate...
    - mean #wins, standard deviation, standard error
    - sample size vs error?
- More general, how to characterise a sample (distribution)
    - mean, median, mode
    - standard deviation, standard error
    - quantiles and box plots
- How to calculate them with pen & pape for small examples?!?
- Appendix: how to calculate the standard quantities in Excel, Tableau, Python

How to calculate these quantities
- mean
- standard deviation
- standard error

[no, not this mean](https://en.wikipedia.org/wiki/File:Taylor_Swift_-_Mean.png#/media/File:Taylor_Swift_-_Mean.png)

if you fount this funny, you may like [this list.](https://www.isg.rhul.ac.uk/~sdg/mathsongs.html) You may also have issues.

In [1]:
import altair as alt
import helpers.plotting as pt
pt.enable_slide_theme()
pt.import_lato_font_in_notebook()

In [2]:
# Simple example for statistical measures:
# drawing coins from a pocket

In [3]:
import pandas as pd
import numpy as np

In [4]:
from helpers.svg_wrapper import SVGImg

In [32]:
# test: simple example to calculate mentally
#coins_in_pocket = [1,2,2,5,20]
coins_in_pocket = [1,1,1,2,5,5,20]
#coins_in_pocket = [1,1,1,2,2,2,2,5,20]
#coins_in_pocket = [5,5,10,20,50]
observations = pd.DataFrame(
    coins_in_pocket,
    columns = ['value'],
    index = pd.Index(range(1,len(coins_in_pocket)+1), name='draw')
)

In [33]:
observations.std() / np.sqrt(len(observations)), observations.var()

(value    2.591194
 dtype: float64,
 value    47.0
 dtype: float64)

In [34]:
observations.describe()

,value
count,7.000000
mean,5.000000
std,6.855655
min,1.000000
25%,1.000000
50%,2.000000
75%,5.000000
max,20.000000


In [35]:
observations.sum(), observations.std() / np.sqrt(len(observations))

(value    35
 dtype: int64,
 value    2.591194
 dtype: float64)

## How to characterise a sample of quantitative observations?

- **Central tendency**: where are the observed values?
- **Dispersion**: how far apart are the observed values?

### For example, imagine drawing coins from a pocket

- How much is each new coin worth?
- How certain are we about that value?

Let's start with a sample of 7 coins...

### Measures of central tendency

In [36]:
SVGImg('images/coins_central_tendency.svg', width='70%', output_dir='slides')

\*If there is an even number of observations in our sample, <br>the median is in between the two central ones.
- median({1,1,1,**2**,5,5,20}) = 2
- median({1,1,**2**,**5**,5,20}) = (2+5) / 2 = 3.5

See also https://en.wikipedia.org/wiki/Central_tendency

In [ ]:
# test: interactive distribution parameters
plot_width=400

input_mean = alt.binding(
    input='range',
    min=-10,
    max=10,
    step=.1,
    name='mean'
)
input_std = alt.binding(
    input='range',
    min=0.1,
    max=5,
    step=.1,
    name='std'
)
mean_selection = alt.selection_single(
    bind=input_mean,
    init={'mean': 0}
)
std_selection = alt.selection_single(
    bind=input_std,
    init={'std': 1}
)
scales = alt.selection_interval(bind='scales')
                                
data = alt.sequence(0, 100, as_='t')
chart = alt.Chart(data).add_selection(
    scales, std_selection, mean_selection
).transform_calculate(
  mean = mean_selection['mean'],
  std  = std_selection['std'],
).transform_calculate(
    t='datum.t',
    x='sampleNormal(+datum.mean,+datum.std)', # the plus casts to number
)
scale = alt.Scale(domain=[-10,10])
hist = chart.mark_bar().encode(
    alt.X("x:Q", bin=True, scale=scale, axis=None),
    y='count()',
).properties(
    width=plot_width
)
ticks = chart.mark_tick().encode(
    x=alt.X('x:Q', scale=scale)
).properties(
    view=alt.ViewConfig(strokeWidth=0), width=plot_width)

(hist & ticks)

In [ ]:
import numpy as np
#from helpers.cards import draw_cards